In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
from sklearn import linear_model
# from __future__ import division

%matplotlib inline
seaborn.set()
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

G:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
G:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,
G:\Anaconda3\lib\site

In [2]:
# GDP周同比增长率
f_GDP = open(r'weekly_tracker.csv',encoding = "gbk")
data_GDP = pd.read_csv(f_GDP)
# 处理时间格式
data_GDP['date'] = pd.to_datetime(data_GDP['date'])

In [3]:
# 政策数据导入和初步处理
f_policy = open(r'OxCGRT_latest.csv',encoding='utf-8')
data_policy = pd.read_csv(f_policy)
# 处理时间格式
data_policy['Date'] = data_policy['Date'].astype(str)
data_policy['Date'] = pd.to_datetime(data_policy['Date'])
# 只取国家级别的政策信息
data_policy = data_policy[data_policy['Jurisdiction'] == 'NAT_TOTAL']

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 国家名统一
data_policy['CountryName'][data_policy['CountryName']=='Democratic Republic of Congo'] = 'Congo (Kinshasa)'
data_policy['CountryName'][data_policy['CountryName']=='Congo'] = 'Congo (Brazzaville)'
# data_policy['CountryName'][data_policy['CountryName']=='Slovak Republic'] = 'Slovakia'

# 国家名统一
data_GDP['region'][data_GDP['region']=='Korea'] = 'South Korea'
# data_GDP['region'][data_GDP['region']=='Slovak Republic'] = 'Slovakia'

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:
# 对政策指标的二值化更改
# 封闭措施
# C1
data_policy['C1'] = 0
data_policy['C1'][data_policy['C1_School closing']<=1.0] = 0
data_policy['C1'][data_policy['C1_School closing']>=2.0] = 1
# C2
data_policy['C2'] = 0
data_policy['C2'][data_policy['C2_Workplace closing']<=1.0] = 0
data_policy['C2'][data_policy['C2_Workplace closing']>=2.0] = 1
# C3
data_policy['C3'] = 0
data_policy['C3'][data_policy['C3_Cancel public events']>=1.0] = 1
# C4
data_policy['C4'] = 0
data_policy['C4'][data_policy['C4_Restrictions on gatherings']<=3.0] = 0
data_policy['C4'][data_policy['C4_Restrictions on gatherings']>=4.0] = 1
# C5
data_policy['C5'] = 0
data_policy['C5'][data_policy['C5_Close public transport']<=1.0] = 0
data_policy['C5'][data_policy['C5_Close public transport']>=2.0] = 1
# C6
data_policy['C6'] = 0
data_policy['C6'][data_policy['C6_Stay at home requirements']<=1.0] = 0
data_policy['C6'][data_policy['C6_Stay at home requirements']>=2.0] = 1
# C7
data_policy['C7'] = 0
data_policy['C7'][data_policy['C7_Restrictions on internal movement']<=1.0] = 0
data_policy['C7'][data_policy['C7_Restrictions on internal movement']>=2.0] = 1
# C8
data_policy['C8'] = 0
data_policy['C8'][data_policy['C8_International travel controls']<=1.0] = 0
data_policy['C8'][data_policy['C8_International travel controls']>=2.0] = 1

# 经济政策
# E1
data_policy['E1'] = 0
data_policy['E1'][data_policy['E1_Income support']>=1.0] = 1
# E2
data_policy['E2'] = 0
data_policy['E2'][data_policy['E2_Debt/contract relief']<=1.0] = 0
data_policy['E2'][data_policy['E2_Debt/contract relief']>=2.0] = 1
# E3
data_policy['E3'] = 0
# data_policy['E3'][data_policy['E3_Fiscal measures']<1e+07] = 0
data_policy['E3'][data_policy['E3_Fiscal measures']>0] = 1
# E4
data_policy['E4'] = 0
# data_policy['E4'][data_policy['E4_International support']<1e+05] = 0
data_policy['E4'][data_policy['E4_International support']>0] = 1

# 卫生系统政策
# H1
data_policy['H1'] = 0
data_policy['H1'][data_policy['H1_Public information campaigns']<=1.0] = 0
data_policy['H1'][data_policy['H1_Public information campaigns']>=2.0] = 1
# H2
data_policy['H2'] = 0
data_policy['H2'][data_policy['H2_Testing policy']<=1.0] = 0
data_policy['H2'][data_policy['H2_Testing policy']>=2.0] = 1
# H3
data_policy['H3'] = 0
data_policy['H3'][data_policy['H3_Contact tracing']<=1.0] = 0
data_policy['H3'][data_policy['H3_Contact tracing']>=2.0] = 1
# H4
data_policy['H4'] = 0
# data_policy['H4'][data_policy['H4_Emergency investment in healthcare']<1e+06] = 0
data_policy['H4'][data_policy['H4_Emergency investment in healthcare']>0] = 1
# H5
data_policy['H5'] = 0
# data_policy['H5'][data_policy['H5_Investment in vaccines']<1e+06] = 0
data_policy['H5'][data_policy['H5_Investment in vaccines']>0] = 1
# H6
data_policy['H6'] = 0
data_policy['H6'][data_policy['H6_Facial Coverings']<=2.0] = 0
data_policy['H6'][data_policy['H6_Facial Coverings']>=3.0] = 1
# H7
data_policy['H7'] = 0
data_policy['H7'][data_policy['H7_Vaccination policy']<=3.0] = 0
data_policy['H7'][data_policy['H7_Vaccination policy']>=4.0] = 1

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\zay\AppD

In [6]:
# 建立字典，对应政策代号及其中英文名
dic_policyEng = dict()
dic_policyChi = dict()

# 封闭措施
dic_policyEng['C1'] = 'School closing'
dic_policyChi['C1'] = '关闭学校'
dic_policyEng['C2'] = 'Workplace closing'
dic_policyChi['C2'] = '关闭工作场所'
dic_policyEng['C3'] = 'Cancel public events'
dic_policyChi['C3'] = '取消公共活动'
dic_policyEng['C4'] = 'Restrictions on gatherings'
dic_policyChi['C4'] = '限制集会'
dic_policyEng['C5'] = 'Close public transport'
dic_policyChi['C5'] = '关闭公共交通'
dic_policyEng['C6'] = 'Stay at home requirements'
dic_policyChi['C6'] = '居家要求'
dic_policyEng['C7'] = 'Restrictions on internal movement'
dic_policyChi['C7'] = '国内移动限制'
dic_policyEng['C8'] = 'International travel controls'
dic_policyChi['C8'] = '国际旅行控制'

# 经济政策
dic_policyEng['E1'] = 'Income support'
dic_policyChi['E1'] = '收入支持'
dic_policyEng['E2'] = 'Debt/contract relief'
dic_policyChi['E2'] = '债务/合同放宽'
dic_policyEng['E3'] = 'Fiscal measures'
dic_policyChi['E3'] = '财政措施'
dic_policyEng['E4'] = 'International support'
dic_policyChi['E4'] = '给予国际支持'

# 卫生系统政策
dic_policyEng['H1'] = 'Public information campaigns'
dic_policyChi['H1'] = '公共信息宣传'
dic_policyEng['H2'] = 'Testing policy'
dic_policyChi['H2'] = '检测政策'
dic_policyEng['H3'] = 'Contact tracing'
dic_policyChi['H3'] = '接触者追踪'
dic_policyEng['H4'] = 'Emergency investment in healthcare'
dic_policyChi['H4'] = '医疗保健紧急投资'
dic_policyEng['H5'] = 'Investment in vaccines'
dic_policyChi['H5'] = '疫苗投资'
dic_policyEng['H6'] = 'Facial Coverings'
dic_policyChi['H6'] = '口罩政策'
dic_policyEng['H7'] = 'Vaccination Policy'
dic_policyChi['H7'] = '疫苗接种政策'

ls_code = list(dic_policyEng.keys())
ls_code

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'E1',
 'E2',
 'E3',
 'E4',
 'H1',
 'H2',
 'H3',
 'H4',
 'H5',
 'H6',
 'H7']

In [7]:
# 此函数用于给出政策是否引入和取消的指标
def genFlags(policySeries):
    Length = policySeries.shape[0]
    introduceSeries = pd.Series(np.zeros(Length))
    liftSeries = pd.Series(np.zeros(Length))
    if policySeries.iloc[0] == 1:
        introduceSeries.iloc[0] = 1
    else:
        introduceSeries.iloc[0] = 0
    liftSeries.iloc[0] = 0
    for i in range(1,Length):
        if policySeries.iloc[i-1] == 0 and policySeries.iloc[i] == 1:   # 政策从0变为1，则introduce指标变为1
            introduceSeries.iloc[i] = 1
        elif policySeries.iloc[i-1] ==1 and policySeries.iloc[i] == 0: # 政策从1变为0，则lift指标变为1
            liftSeries.iloc[i] = 1
        else:
            introduceSeries.iloc[i] = 0
            liftSeries.iloc[i] = 0
    return introduceSeries,liftSeries

In [8]:
# 提取主要信息
data_brief = data_policy[['CountryName','Date']]
for code in ls_code:
    data_brief[code] = data_policy[code]
data_brief

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,CountryName,Date,C1,C2,C3,C4,C5,C6,C7,C8,...,E2,E3,E4,H1,H2,H3,H4,H5,H6,H7
0,Aruba,2020-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aruba,2020-01-02,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aruba,2020-01-03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aruba,2020-01-04,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aruba,2020-01-05,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110428,Zimbabwe,2021-01-23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110429,Zimbabwe,2021-01-24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110430,Zimbabwe,2021-01-25,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110431,Zimbabwe,2021-01-26,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
for code in ['E3','E4','H4','H5','H7']:
    ls_code.remove(code)

In [10]:
# 当天有无政策
data_brief['policyExists'] = 0
for code in ls_code:
    data_brief['policyExists'] += data_brief[code]
data_brief['policyExists'][data_brief['policyExists']>1] = 1

# 对每个政策判断其是否引入和取消，28天内有无该政策
data_prepare = pd.DataFrame()
# 取GDP表中的国家
ls_country = list(data_GDP['region'].drop_duplicates())
for country in ls_country:
    subdata = data_brief[data_brief['CountryName']==country]
    # 时间范围
    startDate = pd.to_datetime('2020-01-22')
    endDate = pd.to_datetime('2020-12-27')
    subdata = subdata[(subdata['Date']>=startDate)&(subdata['Date']<=endDate)]
    subdata = subdata.reset_index(drop=True)

    # 改成周度数据
    subdata.index = subdata['Date']
    subdata = subdata.resample('W').max()
    subdata = subdata.reset_index(drop=True)

    Length = subdata.shape[0]

    for code in ls_code:
        # 政策是否引入或放宽
        introduceSeries,liftSeries = genFlags(subdata[code])
        introduceSeries = introduceSeries.reset_index(drop=True)
        liftSeries = liftSeries.reset_index(drop=True)
        subdata[code+'Introduce'] = introduceSeries
        subdata[code+'Lift'] = liftSeries
        
        # 8周内有无该政策
        subdata[code+'_8W'] = 0
        for i in range(Length):
            end = min(i+8,Length)
            subdata[code+'_8W'].iloc[i] = subdata[code].iloc[i:end].sum()
        # 4周内有无该政策
        subdata[code+'_4W'] = 0
        for i in range(Length):
            end = min(i+4,Length)
            subdata[code+'_4W'].iloc[i] = subdata[code].iloc[i:end].sum()
        
    # 8周内都没有任何政策
    subdata['Policies_8W'] = 0
    for i in range(Length):
        end = min(i+8,Length)
        subdata['Policies_8W'].iloc[i] = subdata['policyExists'].iloc[i:end].sum()

    # 4周内都没有任何政策
    subdata['Policies_4W'] = 0
    for i in range(Length):
        end = min(i+4,Length)
        subdata['Policies_4W'].iloc[i] = subdata['policyExists'].iloc[i:end].sum()

    data_prepare = data_prepare.append(subdata)
data_brief = data_prepare.copy()
data_brief = data_brief.reset_index(drop=True)
data_brief

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

,CountryName,Date,C1,C2,C3,C4,C5,C6,C7,C8,...,H3Introduce,H3Lift,H3_8W,H3_4W,H6Introduce,H6Lift,H6_8W,H6_4W,Policies_8W,Policies_4W
0,Argentina,2020-01-26,0,0,0,0,0,0,0,0,...,0.0,0.0,2,0,0.0,0.0,0,0,8,4
1,Argentina,2020-02-02,0,0,0,0,0,0,0,0,...,0.0,0.0,3,0,0.0,0.0,0,0,8,4
2,Argentina,2020-02-09,0,0,0,0,0,0,0,0,...,0.0,0.0,4,0,0.0,0.0,0,0,8,4
3,Argentina,2020-02-16,0,0,0,0,0,0,0,0,...,0.0,0.0,5,1,0.0,0.0,0,0,8,4
4,Argentina,2020-02-23,0,0,0,0,0,0,0,0,...,0.0,0.0,6,2,0.0,0.0,0,0,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,United States,2020-11-29,1,1,1,1,0,1,1,1,...,0.0,0.0,0,0,0.0,0.0,5,4,5,4
2201,United States,2020-12-06,1,1,1,1,0,1,1,1,...,0.0,0.0,0,0,0.0,0.0,4,4,4,4
2202,United States,2020-12-13,1,1,1,1,0,1,1,1,...,0.0,0.0,0,0,0.0,0.0,3,3,3,3
2203,United States,2020-12-20,1,1,1,1,0,1,1,1,...,0.0,0.0,0,0,0.0,0.0,2,2,2,2


In [11]:
# 加入GDP信息
data_prepare = pd.DataFrame()
for country in ls_country:
    subdata_policy = data_brief[data_brief['CountryName']==country].reset_index(drop=True)

    subdata_GDP = data_GDP[data_GDP['region']==country]
    subdata_GDP = subdata_GDP[(subdata_GDP['date']>=startDate)&(subdata_GDP['date']<=endDate)]
    Series_GDP = subdata_GDP['Tracker (yoy)'].reset_index(drop=True)

    subdata_policy['GDP_Growth'] = Series_GDP
    data_prepare = data_prepare.append(subdata_policy)

data_brief = data_prepare.copy()
data_brief

,CountryName,Date,C1,C2,C3,C4,C5,C6,C7,C8,...,H3Lift,H3_8W,H3_4W,H6Introduce,H6Lift,H6_8W,H6_4W,Policies_8W,Policies_4W,GDP_Growth
0,Argentina,2020-01-26,0,0,0,0,0,0,0,0,...,0.0,2,0,0.0,0.0,0,0,8,4,-0.560012
1,Argentina,2020-02-02,0,0,0,0,0,0,0,0,...,0.0,3,0,0.0,0.0,0,0,8,4,2.016658
2,Argentina,2020-02-09,0,0,0,0,0,0,0,0,...,0.0,4,0,0.0,0.0,0,0,8,4,-1.850084
3,Argentina,2020-02-16,0,0,0,0,0,0,0,0,...,0.0,5,1,0.0,0.0,0,0,8,4,1.737264
4,Argentina,2020-02-23,0,0,0,0,0,0,0,0,...,0.0,6,2,0.0,0.0,0,0,8,4,-0.013552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,United States,2020-11-29,1,1,1,1,0,1,1,1,...,0.0,0,0,0.0,0.0,5,4,5,4,-2.595297
45,United States,2020-12-06,1,1,1,1,0,1,1,1,...,0.0,0,0,0.0,0.0,4,4,4,4,-2.687376
46,United States,2020-12-13,1,1,1,1,0,1,1,1,...,0.0,0,0,0.0,0.0,3,3,3,3,-2.414742
47,United States,2020-12-20,1,1,1,1,0,1,1,1,...,0.0,0,0,0.0,0.0,2,2,2,2,-2.256831


In [12]:
data_brief.to_csv('政策与GDP增长率.csv',index=False,sep=',',encoding="utf_8_sig")

data_brief['CountryName'].drop_duplicates().shape[0]